# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [1]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd

try:
    from scipy.linalg import solve_discrete_are
    _HAS_SCIPY = True
except Exception:
    _HAS_SCIPY = False

def robust_load_tsv_matrix(path):
    # Try reading with header=None first
    df = pd.read_csv(path, sep="\t")
    # If there's an 'index' or unnamed first column that looks like an index, drop it
    possible_index_cols = [c for c in df.columns if str(c).lower() in ("index", "unnamed: 0", "")]
    if possible_index_cols:
        df = df.drop(columns=possible_index_cols)
    # Convert everything to numeric where possible
    for c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    # Drop rows/cols that are fully NaN (in case of trailing tabs)
    df = df.dropna(axis=0, how="all").dropna(axis=1, how="all")
    values = df.values.astype(float)
    return values

A = robust_load_tsv_matrix("model-A.tsv")
B = robust_load_tsv_matrix("model-B.tsv")
Q = robust_load_tsv_matrix("cost-Q.tsv")
R = robust_load_tsv_matrix("cost-R.tsv")

# Validate shapes
n = A.shape[0]
assert A.shape[0] == A.shape[1], f"A must be square, got {A.shape}"
assert B.shape[0] == n, f"B rows must match A, got {B.shape} vs A {A.shape}"
assert Q.shape == (n, n), f"Q must be {n}x{n}, got {Q.shape}"
assert R.shape[0] == R.shape[1], f"R must be square, got {R.shape}"

# Solve DARE and compute K
if _HAS_SCIPY:
    P = solve_discrete_are(A, B, Q, R)
else:
    P = Q.copy()
    for _ in range(20000):
        BtPB = B.T @ P @ B
        inv = np.linalg.inv(R + BtPB)
        P_next = A.T @ P @ A - A.T @ P @ B @ inv @ B.T @ P @ A + Q
        if np.allclose(P_next, P, atol=1e-12, rtol=1e-12):
            break
        P = P_next

K = np.linalg.inv(B.T @ P @ B + R) @ (B.T @ P @ A)



shapes = {
    "A": A.shape,
    "B": B.shape,
    "Q": Q.shape,
    "R": R.shape,
    "K": K.shape,
}
shapes

{'A': (3, 3), 'B': (3, 1), 'Q': (3, 3), 'R': (1, 1), 'K': (1, 3)}

Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [3]:
# YOUR CHANGES HERE

# Save K and display
K_df = pd.DataFrame(K)
K_df_named = K_df.copy()
K_df_named.columns = ["x", "v", "a"]

K_intended_path = "control-K-intended.tsv"
K_df_named.to_csv(K_intended_path, sep="\t", index=False)

K_intended_path


'control-K-intended.tsv'

Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [4]:
# YOUR CHANGES HERE

qc_log = pd.read_csv("qc-log.tsv", sep="\t")

qc_log.columns.tolist()

['index', 'x', 'v', 'a', 'u']

Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [5]:
# YOUR CHANGES HERE


K_df_named = pd.read_csv("control-K-intended.tsv", sep="\t")
K_vec = K_df_named.iloc[0].values.reshape(1, -1)  

X = qc_log[["x", "v", "a"]].values.T  
u_check = -(K_vec @ X).flatten()

qc_check = pd.DataFrame({
    "index": qc_log["index"],
    "u_check": u_check
})

qc_check_path = "qc-check.tsv"
qc_check.to_csv(qc_check_path, sep="\t", index=False)


qc_check_path

'qc-check.tsv'

Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [6]:
# YOUR CHANGES HERE

from sklearn.linear_model import LinearRegression

X = qc_log[["x", "v", "a"]].values
u_actual = qc_log["u"].values

model = LinearRegression(fit_intercept=False)
model.fit(X, -u_actual) 

K_actual = model.coef_.reshape(1, -1)


Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [8]:
# YOUR CHANGES HERE
K_actual_df = pd.DataFrame(K_actual, columns=["x", "v", "a"])

K_actual_final_path = "control-K-actual.tsv"
K_actual_df.to_csv(K_actual_final_path, sep="\t", index=False)

K_actual_final_path

'control-K-actual.tsv'

Submit "control-k-actual.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [9]:
# YOUR CHANGES HERE


qc_train = pd.read_csv("qc-train.tsv", sep="\t")

qc_train.columns.tolist()

['index', 'x', 'v', 'a', 'u']

In [10]:

# Prepare x_t, u_t, x_{t+1}
X_t = qc_train[["x", "v", "a"]].iloc[:-1].values  # current states
U_t = qc_train[["u"]].iloc[:-1].values            # current actions
X_next = qc_train[["x", "v", "a"]].iloc[1:].values  # next states

# Combine [X_t, U_t] for regression
Z = np.hstack([X_t, U_t])  # shape (n_samples, 4)

# Fit for each next-state dimension separately (multivariate regression)
model = LinearRegression(fit_intercept=False)
model.fit(Z, X_next)

# Extract coefficients
coeffs = model.coef_  # shape (3, 4)
A_new = coeffs[:, :3]
B_new = coeffs[:, 3:].reshape(-1, 1)

# Save results
A_new_df = pd.DataFrame(A_new)
B_new_df = pd.DataFrame(B_new)

A_path = "model-A-new.tsv"
B_path = "model-B-new.tsv"

A_new_df.to_csv(A_path, sep="\t", header=False, index=False)
B_new_df.to_csv(B_path, sep="\t", header=False, index=False)


Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [12]:
# YOUR CHANGES HERE

A_new_df = pd.read_csv("model-A-new.tsv", sep="\t", header=None)
B_new_df = pd.read_csv("model-B-new.tsv", sep="\t", header=None)

# Assign column names x, v, a
A_new_df.columns = ["x", "v", "a"]

# Save again in same paths
A_new_df.to_csv("model-A-new.tsv", sep="\t", index=False)


In [13]:
B_new_df = pd.read_csv("model-B-new.tsv", sep="\t", header=0)

B_new_df.columns = ["u"]
B_new_df.insert(0, "index", range(len(B_new_df)))

B_new_df.to_csv("model-B-new.tsv", sep="\t", index=False)


In [14]:

# Load training dataset
qc_train = pd.read_csv("qc-train.tsv", sep="\t")

# Create consecutive pairs
X_t = qc_train[["x", "v", "a"]].iloc[:-1].values
U_t = qc_train[["u"]].iloc[:-1].values
X_next = qc_train[["x", "v", "a"]].iloc[1:].values

# Fit model: X_next = [A B] * [X_t; U_t]
Z = np.hstack([X_t, U_t])
model = LinearRegression(fit_intercept=False)
model.fit(Z, X_next)

# Extract matrices
coeffs = model.coef_
A_new = coeffs[:, :3]
B_new = coeffs[:, 3:].reshape(3, 1)

# Build DataFrames
A_new_df = pd.DataFrame(A_new, columns=["x", "v", "a"])
B_new_df = pd.DataFrame({
    "index": ["x", "v", "a"],
    "u": B_new.flatten()
})

# Save both matrices
A_new_df.to_csv("model-A-new.tsv", sep="\t", index=False)
B_new_df.to_csv("model-B-new.tsv", sep="\t", index=False)


Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.